# Data Wrangling

In [4]:
import pandas as pd
import numpy as np

df=pd.read_csv("AusApparalSales4thQrt2020.csv")
print("Head:")
print(df.head)

print("\nDataset Info:")
df.info()

print("\nSummary Stats:")
display(df.describe(include='all'))


print("\nMissing Values:")
print(df.isna().sum())


print("\nValid Entries:")
print(df.notna().sum())


print("\nDuplicate rows:", df.duplicated().sum())


Head:
<bound method NDFrame.head of              Date        Time State     Group  Unit  Sales
0      1-Oct-2020     Morning    WA      Kids     8  20000
1      1-Oct-2020     Morning    WA       Men     8  20000
2      1-Oct-2020     Morning    WA     Women     4  10000
3      1-Oct-2020     Morning    WA   Seniors    15  37500
4      1-Oct-2020   Afternoon    WA      Kids     3   7500
...           ...         ...   ...       ...   ...    ...
7555  30-Dec-2020   Afternoon   TAS   Seniors    14  35000
7556  30-Dec-2020     Evening   TAS      Kids    15  37500
7557  30-Dec-2020     Evening   TAS       Men    15  37500
7558  30-Dec-2020     Evening   TAS     Women    11  27500
7559  30-Dec-2020     Evening   TAS   Seniors    13  32500

[7560 rows x 6 columns]>

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7560 entries, 0 to 7559
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    7560 non-null   object
 

,Date,Time,State,Group,Unit,Sales
count,7560,7560,7560,7560,7560.000000,7560.000000
unique,90,3,7,4,NaN,NaN
top,1-Oct-2020,Morning,WA,Kids,NaN,NaN
freq,84,2520,1080,1890,NaN,NaN
mean,NaN,NaN,NaN,NaN,18.005423,45013.558201
std,NaN,NaN,NaN,NaN,12.901403,32253.506944
min,NaN,NaN,NaN,NaN,2.000000,5000.000000
25%,NaN,NaN,NaN,NaN,8.000000,20000.000000
50%,NaN,NaN,NaN,NaN,14.000000,35000.000000
75%,NaN,NaN,NaN,NaN,26.000000,65000.000000



Missing Values:
Date     0
Time     0
State    0
Group    0
Unit     0
Sales    0
dtype: int64

Valid Entries:
Date     7560
Time     7560
State    7560
Group    7560
Unit     7560
Sales    7560
dtype: int64

Duplicate rows: 0


In [5]:
df = df.dropna(subset=['State', 'Group'])

df['Sales'] = df['Sales'].fillna(df['Sales'].median())
df['Unit'] = df['Unit'].fillna(df['Unit'].median())

df = df[(df['Sales'] >= 0) & (df['Unit'] >= 0)]

df = df.drop_duplicates()

In [6]:
df['Sales-N1'] = (df['Sales'] - df['Sales'].min()) / (df['Sales'].max() - df['Sales'].min())

df['Unit-N1'] = (df['Unit'] - df['Unit'].min()) / (df['Unit'].max() - df['Unit'].min())

print("Head after normalization:")
display(df.head())

print("\nPost-normalization min & max for Sales and Units:")
print(df[['Sales-N1', 'Unit-N1']].agg(['min', 'max']))


Head after normalization:


,Date,Time,State,Group,Unit,Sales,Sales-N1,Unit-N1
0,1-Oct-2020,Morning,WA,Kids,8,20000,0.095238,0.095238
1,1-Oct-2020,Morning,WA,Men,8,20000,0.095238,0.095238
2,1-Oct-2020,Morning,WA,Women,4,10000,0.031746,0.031746
3,1-Oct-2020,Morning,WA,Seniors,15,37500,0.206349,0.206349
4,1-Oct-2020,Afternoon,WA,Kids,3,7500,0.015873,0.015873



Post-normalization min & max for Sales and Units:
     Sales-N1  Unit-N1
min       0.0      0.0
max       1.0      1.0


In [7]:
# State-level totals
state_total = df.groupby('State', as_index=False).agg(Sales_Total_State=('Sales', 'sum'))

# Merge back to each row, then compute contribution %
df = df.merge(state_total, on='State', how='left')
df['Sales_Pct_of_State'] = (df['Sales'] / df['Sales_Total_State']).round(4)

# Similarly, add Group-within-State context
state_group_total = (
    df.groupby(['State','Group'], as_index=False).agg(Sales_Total_StateGroup=('Sales','sum'))
)
df = df.merge(state_group_total, on=['State','Group'], how='left')
df['Sales_Pct_of_StateGroup'] = (df['Sales'] / df['Sales_Total_StateGroup']).round(4)

display(df.head())


,Date,Time,State,Group,Unit,Sales,Sales-N1,Unit-N1,Sales_Total_State,Sales_Pct_of_State,Sales_Total_StateGroup,Sales_Pct_of_StateGroup
0,1-Oct-2020,Morning,WA,Kids,8,20000,0.095238,0.095238,22152500,0.0009,5625000,0.0036
1,1-Oct-2020,Morning,WA,Men,8,20000,0.095238,0.095238,22152500,0.0009,5752500,0.0035
2,1-Oct-2020,Morning,WA,Women,4,10000,0.031746,0.031746,22152500,0.0005,5262500,0.0019
3,1-Oct-2020,Morning,WA,Seniors,15,37500,0.206349,0.206349,22152500,0.0017,5512500,0.0068
4,1-Oct-2020,Afternoon,WA,Kids,3,7500,0.015873,0.015873,22152500,0.0003,5625000,0.0013


# Data Analysis

In [14]:
sales_mean = df['Sales'].mean()
sales_median = df['Sales'].median()
sales_mode = df['Sales'].mode()[0]
sales_std = df['Sales'].std()

print("Sales Statistics:")
print(f"Mean: {sales_mean:.2f}")
print(f"Median: {sales_median:.2f}")
print(f"Mode: {sales_mode:.2f}")
print(f"Standard Deviation: {sales_std:.2f}")

unit_mean = df['Unit'].mean()
unit_median = df['Unit'].median()
unit_mode = df['Unit'].mode()[0]
unit_std = df['Unit'].std()

print("\nUnit Statistics: ")
print(f"Mean: {unit_mean:.2f}")
print(f"Median: {unit_median:.2f}")
print(f"Mode: {unit_mode:.2f}")
print(f"Standard Deviation: {unit_std:.2f}")


Sales Statistics:
Mean: 45013.56
Median: 35000.00
Mode: 22500.00
Standard Deviation: 32253.51

Unit Statistics: 
Mean: 18.01
Median: 14.00
Mode: 9.00
Standard Deviation: 12.90


# Data Visualization

# Report Generation